In [49]:
import torch
from torch_geometric.datasets import TUDataset, Planetoid, Reddit
import numpy as np
import matplotlib.pyplot as plt

In [50]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 

In [48]:
#Download PROTEINS dataset and save in data
dataset = Planetoid(root="data", name="Pubmed")
data = dataset[0]

#Print information about the dataset
print(f'Dataset: {dataset}')
print('-------------------')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

# Print information about the graph
print(f'\nGraph:')
print('------')
print(f'Training nodes: {sum(data.train_mask).item()}')
print(f'Evaluation nodes: {sum(data.val_mask).item()}')
print(f'Testing nodes: {sum(data.test_mask).item()}')
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')
print(f'Node attributes: {data.train_mask}')
print(f'Node Label: {data.test_mask}')


Dataset: Pubmed()
-------------------
Number of graphs: 1
Number of nodes: 19717
Number of features: 500
Number of classes: 3

Graph:
------
Training nodes: 60
Evaluation nodes: 500
Testing nodes: 1000
Edges are directed: False
Graph has isolated nodes: False
Graph has loops: False
Node attributes: tensor([ True,  True,  True,  ..., False, False, False])
Node Label: tensor([False, False, False,  ...,  True,  True,  True])


In [38]:
from torch_geometric.loader import NeighborLoader
kwargs = {'batch_size': 1024, 'num_workers': 6, 'persistent_workers': True}
train_loader = NeighborLoader(data, input_nodes=data.train_mask, num_neighbors=[25, 10], shuffle=True, **kwargs)

In [39]:
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F
from torch.nn import Dropout, Linear

class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, aggregator_type="mean"):
        super().__init__()
        #self.optimizer = torch.optim.Adam(self.parameters())
        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

    def forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i < len(self.convs) - 1:
                x = x.relu_()
                x = F.dropout(x, p=0.5, training=self.training)
        return x

In [40]:
def train(model, optimizer):
    model.train()

    total_loss = total_correct = total_examples = 0
    for batch in train_loader:
        optimizer.zero_grad()
        y = batch.y[:batch.batch_size]
        y_hat = model(batch.x, batch.edge_index)[:batch.batch_size]
        loss = F.cross_entropy(y_hat, y)
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * batch.batch_size
        total_correct += int((y_hat.argmax(dim=-1) == y).sum())
        total_examples += batch.batch_size
        
    return total_loss / total_examples, total_correct / total_examples

In [42]:

model = SAGE(dataset.num_features, 256, dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
for epoch in range(1, 101):
    loss, acc = train(model, optimizer)
    print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')


Epoch 01, Loss: 1.1004, Approx. Train: 0.3333
Epoch 02, Loss: 1.0446, Approx. Train: 0.8667
Epoch 03, Loss: 0.9582, Approx. Train: 0.9500
Epoch 04, Loss: 0.8355, Approx. Train: 0.9667
Epoch 05, Loss: 0.6964, Approx. Train: 0.9500
Epoch 06, Loss: 0.5512, Approx. Train: 0.9833
Epoch 07, Loss: 0.4149, Approx. Train: 0.9500
Epoch 08, Loss: 0.3030, Approx. Train: 0.9667
Epoch 09, Loss: 0.2050, Approx. Train: 0.9667
Epoch 10, Loss: 0.1455, Approx. Train: 0.9833
Epoch 11, Loss: 0.0939, Approx. Train: 0.9833
Epoch 12, Loss: 0.0605, Approx. Train: 1.0000
Epoch 13, Loss: 0.0362, Approx. Train: 1.0000
Epoch 14, Loss: 0.0267, Approx. Train: 1.0000
Epoch 15, Loss: 0.0159, Approx. Train: 1.0000
Epoch 16, Loss: 0.0089, Approx. Train: 1.0000
Epoch 17, Loss: 0.0093, Approx. Train: 1.0000
Epoch 18, Loss: 0.0039, Approx. Train: 1.0000
Epoch 19, Loss: 0.0037, Approx. Train: 1.0000
Epoch 20, Loss: 0.0021, Approx. Train: 1.0000
Epoch 21, Loss: 0.0022, Approx. Train: 1.0000
Epoch 22, Loss: 0.0019, Approx. Tr